In [1]:
import pandas
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from openai import OpenAI
import utils

In [2]:
disease='SLE'
symp_type='formal'
#model_old='chatgpt-3.5'
#model_new='chatgpt-4o'
#model_new_name = 'gpt-4o'


model_old='claude-3-opus'
model_new='claude-3.5-sonnet'
model_new_name = 'claude-3-5-sonnet-20240620'
filename=f'data/{disease}-{symp_type}-{model_old}.txt'
output_filename = f'data/{disease}-{symp_type}-{model_new}.txt'
keywords_list = ['sle', 'lupus']

## Run More Complex Models

The goal is to run more complex models and compare against the outputs of the baseline model.

In [3]:

all_patients = utils.process_file(filename)

Patient: number 1, gender female, age 10, weight 54, height 46
Symptom: Arthritis
Symptom: Malar rash
Symptom: Photosensitive rash
Symptom: Nephritis
Symptom: Pericardiitis 
response: 1,*Systemic Lupus Erythematosus (SLE)*,80%,*The combination of arthritis, malar rash, photosensitive rash, nephritis, and pericarditis is highly suggestive of SLE, an autoimmune disorder that can affect multiple organ systems.*
response: 2,*Juvenile Idiopathic Arthritis (JIA)*,10%,*Arthritis is a prominent symptom, and JIA is a group of autoimmune disorders that can cause joint inflammation in children.*
response: 3,*Mixed Connective Tissue Disease (MCTD)*,5%,*MCTD is an autoimmune disorder that can present with symptoms similar to SLE, including arthritis, rash, and organ involvement.*
response: 4,*Vasculitis*,3%,*Vasculitis is an inflammation of blood vessels that can cause various symptoms, including rash, arthritis, and organ involvement.*
response: 5,*Juvenile Dermatomyositis*,2%,*Juvenile Dermatomyo

In [5]:
utils.run_llm_and_parse_response(all_patients[5], model_new_name)

Patient ID: 6


[('Systemic Lupus Erythematosus (SLE)',
  '70',
  ' Matches multiple symptoms including fever, lymphadenopathy, arthritis, and organ involvement (nephritis, cardiovascular, pulmonary); common in young females'),
 ('Juvenile Idiopathic Arthritis (JIA)',
  '60',
  ' Explains arthritis, fever, and lymphadenopathy; can affect multiple organs and is common in children'),
 ('Kawasaki Disease',
  '45',
  ' Can cause fever, lymphadenopathy, and cardiovascular involvement; typically affects children under 5 but can occur in older children'),
 ('Lyme Disease',
  '35',
  ' Can cause fever, arthritis, and affect multiple organ systems; less likely due to the extent of organ involvement but possible if left untreated'),
 ('Lymphoma',
  '25',
  ' Could explain lymphadenopathy, hepatosplenomegaly, and systemic symptoms; less likely due to age but should be considered given the presentation')]

In [6]:
resps = [ utils.run_llm_and_parse_response(p, model_new_name) for p in all_patients]

Patient ID: 1
Patient ID: 2
Patient ID: 3
Patient ID: 4
Patient ID: 5
Patient ID: 6
Patient ID: 7
Patient ID: 8
Patient ID: 9
Patient ID: 10
Patient ID: 11
Patient ID: 12
Patient ID: 13
Patient ID: 14
Patient ID: 15
Patient ID: 16
Patient ID: 17
Patient ID: 18
Patient ID: 19
Patient ID: 20
Patient ID: 21
Patient ID: 22
Patient ID: 23
Patient ID: 24
Patient ID: 25
Patient ID: 26
Patient ID: 27
Patient ID: 28
Patient ID: 29
Patient ID: 30
Patient ID: 31
Patient ID: 32
Patient ID: 33
Patient ID: 34
Patient ID: 35
Patient ID: 36
Patient ID: 37
Patient ID: 38
Patient ID: 39
Patient ID: 40
Patient ID: 41
Patient ID: 42
Patient ID: 43
Patient ID: 44
Patient ID: 45
Patient ID: 46
Patient ID: 47
Patient ID: 48
Patient ID: 49
Patient ID: 50
Patient ID: 51
Patient ID: 52
Patient ID: 53
Patient ID: 54
Patient ID: 55
Patient ID: 56
Patient ID: 57
Patient ID: 58
Patient ID: 59
Patient ID: 60
Patient ID: 61
Patient ID: 62
Patient ID: 63
Patient ID: 64
Patient ID: 65
Patient ID: 66
Patient ID: 67
Pati

In [7]:
print(resps)

[[('Systemic Lupus Erythematosus (SLE)', '80', ' Multiple symptoms (malar rash, photosensitivity, arthritis, nephritis, pericarditis) strongly suggest SLE, which can affect children'), ('Juvenile Idiopathic Arthritis (JIA)', '60', ' Arthritis is a primary symptom, and some JIA subtypes can involve skin rashes and organ involvement'), ('Mixed Connective Tissue Disease', '40', ' Combines features of lupus, scleroderma, and polymyositis; can explain multiple system involvement'), ('Lyme Disease', '30', ' Can cause arthritis, rash, and cardiac involvement, though the specific rash types are less typical'), ('Drug-Induced Lupus', '20', ' Similar symptoms to SLE, but less common in children and typically requires long-term medication exposure')], [('Systemic Lupus Erythematosus (SLE)', '75', ' Multiple systemic symptoms including malar rash, photosensitivity, arthritis, and organ involvement (nephritis, pleuritis) are highly suggestive of SLE, especially in a young female.'), ('Juvenile Idio

In [8]:

def dump_response_to_file(filehandle, patient, resp):
    print(f'P: {patient.n}, {patient.gender}, {patient.age}, {patient.weight}, {patient.height}', file=filehandle)
    for s in patient.symptom_list:
        print(f'S:{s}', file=filehandle)
    for (j, r) in enumerate(resp):
        (disease_name, prob, descr) = r
        print(f'R: {j+1}, {disease_name}, {prob}, {descr}', file=filehandle)

# dump responses to output file
fhandle = open(output_filename,'w')
for (pt, resp) in zip(all_patients, resps):
    dump_response_to_file(fhandle, pt, resp)
fhandle.close()

In [9]:
def analyze_responses_for_top_three(resps, keywords_list, k=3):
    num_matches = 0
    for resp in resps:
        for (j, r) in enumerate(resp):
            if (j < k):
                (disease_name, prob, descr) = r
                if utils.approx_matches(keywords_list, [disease_name]):
                    num_matches = num_matches + 1
                    break 
    return num_matches

In [10]:
matches_1 = analyze_responses_for_top_three(resps, keywords_list, k=1)
print(f'Chat GPT 4 is able to find {disease} in top 1 for {matches_1} out of {len(resps)} cases')
matches_3 = analyze_responses_for_top_three(resps, keywords_list, k=3)
print(f'Chat GPT 4 is able to find {disease} in top 3 for {matches_3} out of {len(resps)} cases ')

Chat GPT 4 is able to find SLE in top 1 for 239 out of 249 cases
Chat GPT 4 is able to find SLE in top 3 for 246 out of 249 cases 


In [11]:
not_in_top_3 = utils.select_where_x_is_not_top(all_patients, keywords_list)
print(len(not_in_top_3))

2


In [12]:
chat_gpt_4_responses = [resps[pt.n-1] for pt in not_in_top_3]
matches_1 = analyze_responses_for_top_three(chat_gpt_4_responses, keywords_list, k=1)
print(f'Chat GPT 4 is able to find {disease} in top 1 for {matches_1} out of {len(not_in_top_3)} cases where ChatGPT3.5 fails')
matches_3 = analyze_responses_for_top_three(chat_gpt_4_responses, keywords_list, k=3)
print(f'Chat GPT 4 is able to find {disease} in top 3 for {matches_3} out of {len(not_in_top_3)} cases where ChatGPT3.5 fails ')

Chat GPT 4 is able to find SLE in top 1 for 0 out of 2 cases where ChatGPT3.5 fails
Chat GPT 4 is able to find SLE in top 3 for 0 out of 2 cases where ChatGPT3.5 fails 
